In [54]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account


In [55]:
projectId='pongthorn'
table_id = f"{projectId}.DemoSMartDW.incident"

client = bigquery.Client(project=projectId)

start_date_query='2000-01-01'

In [56]:
try:
    client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
except NotFound:
    raise Exception("Table {} is not found.".format(table_id))

Table pongthorn.DemoSMartDW.incident already exists.


In [57]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
         database='SMartDB', user='postgres',
      password='P@ssw0rd', host='localhost', 
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [58]:
sql_incident="""

select
incident.id as id, incident.incident_no as incident_no,

severity.id as  severity_id,
severity.severity_name as  severity_name,

service_level.sla_name as sla,

product_type.productype_name as product_type,brand.brand_name as brand,
xtype.incident_type_name as incident_type,
status.incident_status_name as status,
service.service_type_name service_type

,(select count(*) from  app_incident_detail  as detail where  detail.incident_master_id=incident.id ) as count_detail

,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as resolved_datetime

,TO_CHAR(incident.updated_at,'YYYY-MM-DD HH24:MI:SS') as updated_at


from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id

inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id

inner join app_inventory as inventory on incident.inventory_id = inventory.id

inner join app_brand as brand on inventory.brand_id = brand.id

inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_sla as service_level on inventory.customer_sla_id = service_level.id
where incident.incident_status_id =4  and incident.updated_at>=%(start_date_param)s

order by incident.incident_datetime desc


"""

In [59]:
dt_imported=datetime.now()
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )

sql_lastImport=f"SELECT max(imported_at) as last_imported from `{table_id}` where open_datetime>='{start_date_query}' "
job_lastImported=client.query(sql_lastImport)
str_lastImported=None
for row in job_lastImported:    
    if row.last_imported is not None: 
        str_lastImported=row.last_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Last Imported DateTime: {str_lastImported}" )

if str_lastImported is not None:
  start_date_query=str_lastImported

print(f"Start Import on update_at of last imported date : {start_date_query}" )

Imported DateTime: 2023-04-17 22:16:00
Last Imported DateTime: 2023-04-03 14:25:56
Start Import on update_at of last imported date : 2023-04-03 14:25:56


In [60]:

dict_params={"start_date_param":start_date_query}
df_all=list_data(sql_incident,dict_params,get_postgres_conn())

if df_all.empty==True:
    print("no transsaction update")
    exit()

In [61]:
print(df_all.info())
df_all.tail(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 56 non-null     int64 
 1   incident_no        56 non-null     object
 2   severity_id        56 non-null     int64 
 3   severity_name      56 non-null     object
 4   sla                56 non-null     object
 5   product_type       56 non-null     object
 6   brand              56 non-null     object
 7   incident_type      56 non-null     object
 8   status             56 non-null     object
 9   service_type       56 non-null     object
 10  count_detail       56 non-null     int64 
 11  open_datetime      56 non-null     object
 12  close_datetime     56 non-null     object
 13  response_datetime  56 non-null     object
 14  resolved_datetime  56 non-null     object
 15  updated_at         56 non-null     object
dtypes: int64(3), object(13)
memory usage: 7.1+ KB


,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,updated_at
51,2323,SR-ES-23-2323,4,Cosmatic,24x7 4Hrs Response Time,Storage,NetApp,Software,Closed,Request,3,2023-02-09 17:30,2023-04-07 11:40,2023-02-09 17:30,2023-04-07 11:40,2023-04-08 19:32:55
52,2464,SR-ES-23-2464,2,Major,24x7 6Hrs Resolution Time,Software,Alfresco,General Incident,Closed,Incident,1,2023-01-25 10:08,2023-01-26 17:00,2023-01-25 10:09,2023-01-26 17:00,2023-04-04 14:26:30
53,2165,SR-ES-23-2165,2,Major,24x7 4Hrs Resolution Time,Hardware,Oracle,General Incident,Closed,Incident,5,2023-01-04 09:00,2023-04-10 16:00,2023-01-04 09:05,2023-04-10 10:00,2023-04-11 12:03:50
54,2041,SR-ES-22-2041,3,Minor,24x7 6Hrs Resolution Time,Storage,NetApp,General Incident,Closed,Request,10,2022-11-14 14:11,2023-02-01 10:00,2022-11-14 14:55,2023-02-01 10:00,2023-04-11 09:59:44
55,131,SR-ES-22-0131,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,OS / Firmware,Closed,Incident,17,2022-01-19 09:59,2023-04-10 11:30,2022-01-19 10:30,2023-04-10 11:30,2023-04-10 12:20:03


In [62]:
# convert object to datetime
dateTimeCols=['open_datetime','response_datetime','resolved_datetime','close_datetime','updated_at']
for col in dateTimeCols:
 df_all[col]=pd.to_datetime(df_all[col], format='%Y-%m-%d %H:%M',errors= 'coerce')



In [63]:
df_all['imported_at']=dt_imported

df_all.dropna(inplace=True)

print(df_all.info())
df_all.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 55
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 55 non-null     int64         
 1   incident_no        55 non-null     object        
 2   severity_id        55 non-null     int64         
 3   severity_name      55 non-null     object        
 4   sla                55 non-null     object        
 5   product_type       55 non-null     object        
 6   brand              55 non-null     object        
 7   incident_type      55 non-null     object        
 8   status             55 non-null     object        
 9   service_type       55 non-null     object        
 10  count_detail       55 non-null     int64         
 11  open_datetime      55 non-null     datetime64[ns]
 12  close_datetime     55 non-null     datetime64[ns]
 13  response_datetime  55 non-null     datetime64[ns]
 14  resolved_dat

,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,updated_at,imported_at
0,2640,SR-ES-23-2640,2,Major,24x7 4Hrs Response Time,Server,HPE,Memory Failure,Closed,Incident,2,2023-04-14 13:08:00,2023-04-17 14:49:00,2023-04-14 13:17:00,2023-04-14 22:00:00,2023-04-17 14:48:42,2023-04-17 22:16:00.464817
1,2633,SR-ES-23-2633,3,Minor,24x7 6Hrs Resolution Time,Software,Alfresco,Software,Closed,Incident,1,2023-04-12 10:30:00,2023-04-12 10:59:00,2023-04-12 10:31:00,2023-04-12 10:59:00,2023-04-12 11:13:14,2023-04-17 22:16:00.464817
2,2639,SR-ES-23-2639,2,Major,24x7 4Hrs Resolution Time,Software,VMWare,Other Failure,Closed,Request,1,2023-04-12 09:00:00,2023-04-12 15:00:00,2023-04-12 09:00:00,2023-04-12 15:00:00,2023-04-17 08:35:29,2023-04-17 22:16:00.464817
3,2632,SR-ES-23-2632,2,Major,24x7 4Hrs Resolution Time,Storage,HPE,General Incident,Closed,Incident,3,2023-04-11 13:38:00,2023-04-12 09:59:00,2023-04-11 13:53:00,2023-04-11 15:30:00,2023-04-17 21:29:58,2023-04-17 22:16:00.464817
4,2623,SR-ES-23-2623,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,General Incident,Closed,Incident,1,2023-04-11 09:40:00,2023-04-11 10:00:00,2023-04-11 09:40:00,2023-04-11 10:00:00,2023-04-11 11:50:06,2023-04-17 22:16:00.464817


In [64]:
def insertDataFrameToBQ(df):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df), "Imported closed incident to bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    insertDataFrameToBQ(df_all)
except Exception as ex:
    raise ex


Total  55 Imported closed incident to bigquery successfully


In [66]:
def load_data_bq(sql:str):
 query_result=client.query(sql)
 dfx=query_result.to_dataframe()
 return dfx

sql_query=f""" 
SELECT * FROM `{table_id}` 
where updated_at>= (SELECT max(updated_at) FROM `{table_id}`)

"""
df=load_data_bq(sql_query)
df


,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,updated_at,imported_at
0,2632,SR-ES-23-2632,2,Major,24x7 4Hrs Resolution Time,Storage,HPE,General Incident,Closed,Incident,3,2023-04-11 13:38:00,2023-04-12 09:59:00,2023-04-11 13:53:00,2023-04-11 15:30:00,2023-04-17 21:29:58,2023-04-17 22:16:00.464817
